In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline


In [2]:
fd_train = pd.read_csv('C:/Users/welcome/Downloads/GL_Hackathon/frequency_domain_features_train.csv')
fd_test = pd.read_csv('C:/Users/welcome/Downloads/GL_Hackathon/frequency_domain_features_test.csv')
td_train = pd.read_csv('C:/Users/welcome/Downloads/GL_Hackathon/time_domain_features_train.csv')
td_test = pd.read_csv('C:/Users/welcome/Downloads/GL_Hackathon/time_domain_features_test.csv')
hr_train = pd.read_csv('C:/Users/welcome/Downloads/GL_Hackathon/heart_rate_non_linear_features_train.csv')
hr_test = pd.read_csv('C:/Users/welcome/Downloads/GL_Hackathon/heart_rate_non_linear_features_test.csv')

In [3]:
df3 = pd.merge(fd_train, td_train, how='outer', on='uuid')
df_train = pd.merge(df3, hr_train, how='outer', on='uuid')

In [4]:
df2 = pd.merge(fd_test, td_test, how='outer', on='uuid')
df_test = pd.merge(df2, hr_test, how='outer', on='uuid')

In [5]:
y_train = df_train.pop('HR')
X_train = df_train.copy(deep = True)
X_test = df_test.copy(deep = True)

In [6]:
X_train.drop(columns= ['datasetId'], inplace=True)
X_test.drop(columns= ['datasetId'], inplace=True)
X_train.drop(columns= ['uuid'], inplace=True)
y_id = X_test.pop('uuid')
X_train.drop(columns= ['SKEW_REL_RR','KURT_REL_RR'], inplace=True)
X_test.drop(columns= ['SKEW_REL_RR','KURT_REL_RR'], inplace=True)
X_train.drop(columns= ['SDSD_REL_RR'], inplace=True) 
X_test.drop(columns= ['SDSD_REL_RR'], inplace=True) 


In [7]:
cols = ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'SD1', 'SD2', 'sampen', 'higuci']

from sklearn.linear_model import Lasso

model = Lasso()

model.fit(X_train[cols], y_train)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [8]:
col2 = pd.DataFrame(model.coef_ * 10000,np.array(cols))
col2.columns = ['val']

final_cols = col2[col2['val']!=0].index

X_train = X_train[final_cols]
X_test = X_test[final_cols]



In [13]:
X_train['condition'] = hr_train['condition']
X_test['condition'] = hr_test['condition']

In [14]:
categorical_columns = X_train.describe(include = ['O']).columns
numerical_columns = X_train.describe().columns

In [15]:
X_train.columns.size, X_test.columns.size

(9, 9)

In [16]:
categorical_columns.size +  numerical_columns.size

9

In [17]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA

pl_n = Pipeline([
                #  ('imputerNaN', SimpleImputer(missing_values=np.nan, strategy='median')),
                #  ('imputer0', SimpleImputer(missing_values=0, strategy='median')),
                 ('std', StandardScaler()),
                 ('power', PowerTransformer())
])

pl_c = Pipeline([
                #  ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                 ('std', OneHotEncoder(handle_unknown='ignore'))
])

from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
                        ('num', pl_n, numerical_columns),
                        ('cat', pl_c, categorical_columns)
])

In [18]:
from sklearn.metrics import mean_absolute_error

In [20]:
from sklearn.ensemble import ExtraTreesRegressor

model = Pipeline([
               ('ct', ct),
               ('lr', ExtraTreesRegressor(n_estimators=500, random_state=1, n_jobs=-1))
])

model.fit(X_train, y_train)

y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)


print('Training Score: ', '%0.8f'%mean_absolute_error(y_train, y_train_predict))

Training Score:  0.00002871


In [21]:
y_test_predict

array([64.70946452, 73.77736252, 69.19811525, ..., 60.94928377,
       77.40858181, 76.59910561])

In [22]:
df = pd.concat([pd.DataFrame(y_id),pd.DataFrame(y_test_predict)], axis=1, ignore_index=True)
df.columns = ['uuid','HR']
df.to_csv('C:/Users/welcome/Downloads/GL_Hackathon/ETR500Submission.csv', index=False)